In [1]:
import torch
import numpy as np
from tqdm import tqdm
from transformers.models.bert.modeling_bert import BertModel, BertConfig
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from transformers import AdamW, get_constant_schedule_with_warmup
from torch import nn
import torch.nn.functional as F
import time
from datetime import timedelta
import random
import argparse
from sklearn import metrics
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, PackedSequence
import re
import copy
from collections import defaultdict
import os
import sys

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def get_time_dif(start_time):
    """
    获取已使用时间
    :param: start_time是任务开始的时间
    :return: 返回整体训练的时间，单位s
    """
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))

In [4]:
def pad_and_truncat(sequence,max_seq_len,tokenizer):
    max_len = max_seq_len - 1
    sequence = '[CLS]' + sequence
    tokens = tokenizer.tokenize(sequence)
    if len(tokens) < max_len:
        tokens = tokens + ['[SEP]']
        tokens = tokens + ['[PAD]']*(max_seq_len - len(tokens))
    else:
        tokens = tokens[:max_len] + ['[SEP]']
    return tokens
  
class Tokenizer4bert:
    def __init__(self,max_seq_len,pretrained_bert_path):
        self.tokenizer = BertTokenizer.from_pretrained(pretrained_bert_path)
        self.max_seq_len = max_seq_len
        
    def text_to_idx(self,text):
        text = pad_and_truncat(text, self.max_seq_len, self.tokenizer)
        sequence = self.tokenizer.convert_tokens_to_ids(text)
        if len(sequence)==0:
            sequence = [0]
        truncat = np.asarray(sequence,dtype = 'int64')
        x = (np.ones(self.max_seq_len)*0).astype('int64')
        x[:len(truncat)] = truncat
        return x

In [5]:
tokenizer = Tokenizer4bert(20, 'bert-base-chinese')

In [6]:
out = tokenizer.text_to_idx('我很爱这部电影虽然它不怎么样！！')

In [7]:
orign_tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [69]:
def load_data(tokenizer, orign_tokenizer,path = '../Data/Dataset/Dataset/dependery.txt', pad_size = 20, orign_size = 150):
    '''
        数据读取，并进行数据格式化
    '''
    all_data = defaultdict(list)
    contexts = []
    labels = []
    origns = []
    dependeries = []
    ans = []
    max_paraghs = -1
    
    print('read data...')
    with open(path,'r',encoding = 'utf-8-sig') as f:
        lines = f.readlines()
        for i, line in tqdm(enumerate(lines)):
            if line == '\n':
                context = [ans[i].strip() for i in range(len(ans)) if i != len(ans) - 1 and i != len(ans) - 2] 
                label = int(ans[-1].strip())
                dependery = ans[-2].strip().split()
                dependery = [int(d) for d in dependery]
                assert len(dependery) == len(context)    
                max_paraghs = max(max_paraghs, len(dependery))
                
                contexts.append(context[:])
                origns.append(''.join(context[:]))
                labels.append(label)
                dependeries.append(dependery[:])
                ans = []
            else:
                ans.append(line.strip())
    
    print('encoding data...')
    origns = orign_tokenizer(origns, max_length = orign_size, padding = True, truncation = True)
    for i, _ in tqdm(enumerate(contexts)):
        context = contexts[i]
        label = labels[i]
        dependery = dependeries[i]
        
        pas = []
        for c in context:
            pas.append(tokenizer.text_to_idx(c))
        if len(pas) < max_paraghs:
            pas.extend([np.zeros(pad_size,dtype = np.int64) for _ in range(max_paraghs-len(pas))])
        else:
            pas = pas[:max_paraghs]
        
        if len(dependery) < max_paraghs:
            dependery += [-2]*(max_paraghs - len(dependery))
        else:
            dependery = dependery[:max_paraghs]
            
        

        all_data['contexts'].append(np.array(pas))
        all_data['dependeries'].append(np.array(dependery))
        all_data['labels'].append(label)
        all_data['input_ids'].append(origns['input_ids'][i])
        all_data['attention_mask'].append(origns['attention_mask'][i])
        all_data['token_type_ids'].append(origns['token_type_ids'][i])
    return all_data, max_paraghs,origns

In [70]:
a,_,o = load_data(tokenizer, orign_tokenizer, '../Data/Dataset/Dataset/test_dependery.txt')

3772it [00:00, 385970.11it/s]

read data...
encoding data...



591it [00:01, 519.47it/s]


In [71]:
class ReDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.data.items()}
        return item
    
    def __len__(self):
        
        return len(self.data['contexts'])

In [72]:
dataset = ReDataset(a)

In [78]:
for i,d in enumerate(dataset):
    assert len(d['input_ids']) == 150

In [79]:
len(d['input_ids'])

150

In [80]:
data_iter = DataLoader(dataset, batch_size = 16)

In [76]:
for batch in data_iter:
    pass

In [82]:
batch['contexts']

tensor([[[ 101, 6206, 1726,  ...,  753, 1923,  102],
         [ 101, 1377, 3221,  ...,    0,    0,    0],
         [ 101, 2769, 1780,  ...,    0,    0,    0],
         ...,
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0]],

        [[ 101, 2769, 6371,  ...,  679, 1963,  102],
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0],
         ...,
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0]],

        [[ 101, 6791, 3191,  ..., 1908,  671,  102],
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0],
         ...,
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0],
         [   0,    0,    0,  ...,    0,    0,    0]],

In [8]:
class Tree(object):
    '''
        构建树节点
    '''
    def __init__(self):
        self.parent = None
        self.children = []
        self.num_children = 0
    
    def add_child(self,child):
        child.parent = self
        self.children.append(child)
        self.num_children += 1
        
    def size(self):
        if getattr(self,'_size'):
            return self._size
        count = 1
        for i in range(self.num_children):
            count += self.children[i].size()
        self._size = count
        return self._size
    
    def depth(self):
        if getattr(self,'_depth'):
            return self._depth
        count = 0
        if self.num_children >0:
            for i in range(self.num_children):
                child_depth = self.children[i].depth()
                if child_depth > count:
                    count = children_depth
            count += 1
        self._depth = count
        return self._depth
    
    def __iter__(self):
        yield self
        for c in self.children:
            for x in c:
                yield x
                
def head_to_tree(head):
    '''
        将依存信息转化成树节点
    '''
    if isinstance(head,list) == False:
        head = head.tolist()
    root = None
    nodes = [Tree() for _ in head]
    for i in range(len(nodes)):
        h = head[i]
        nodes[i].idx = i
        nodes[i].dist = -1
        if h != -1 and h != -2:
            nodes[h].add_child(nodes[i])
        elif h == -1:
            root = nodes[i]
    assert root is not None
    return root

def tree_to_adj(sent_len,tree, directed = False, self_loop = True):
    '''
        将树节点转换成邻接矩阵
    '''
    ret = np.zeros((sent_len,sent_len), dtype = np.float32)
    queue = [tree]
    idx = []
    while len(queue) > 0:
        t, queue = queue[0], queue[1:]
        if not t.children:
            idx += [t.idx]
        
        for c in t.children:
            ret[t.idx,c.idx] = 1
        queue += t.children
    
    if not directed:
        ret = ret + ret.T
    
    if self_loop:
        for i in idx:
            ret[i,i] = 1
    return ret

In [148]:
class DatasetIterater(object):
    '''
        自己建立迭代器对象，主要实现__iter__和__next__两个方法
    '''
    def __init__(self, batches, batch_size):
        self.batch_size = batch_size
        self.batches = batches
        self.n_batches = len(batches) // batch_size
        self.residue = False  # 记录batch数量是否为整数
        if len(batches) % self.n_batches != 0:
            self.residue = True
        self.index = 0

    def __next__(self):#产生下一个元素
        if self.residue and self.index == self.n_batches:
            batches = (torch.LongTensor([c for c,_,_ in self.batches[self.index*self.batch_size:len(self.batches)]]), \
                       torch.LongTensor([h for _,h,_ in self.batches[self.index*self.batch_size:len(self.batches)]])), \
                       torch.LongTensor([p for _,_,p in self.batches[self.index*self.batch_size:len(self.batches)]])
            self.index += 1
            return batches #下一个元素的值

        elif self.index > self.n_batches:
            self.index = 0
            raise StopIteration#next的终止
        else:
            batches = (torch.LongTensor([c for c,_,_ in self.batches[self.index*self.batch_size:(self.index + 1) * self.batch_size]]), \
                       torch.LongTensor([h for _,h,_ in self.batches[self.index*self.batch_size:(self.index + 1) * self.batch_size]])), \
                       torch.LongTensor([p for _,_,p in self.batches[self.index*self.batch_size:(self.index + 1) * self.batch_size]])
            self.index += 1
            return batches #下一个元素的值

    def __iter__(self):#不停放入返回自己
        return self

    def __len__(self):
        if self.residue:
            return self.n_batches + 1
        else:
            return self.n_batches

In [273]:
class PositionwiseFeedForward(nn.Module):
    ''' 相当于一个线性层 '''
    def __init__(self, d_hid, d_inner_hid=None, dropout=0):
        super(PositionwiseFeedForward, self).__init__()
        if d_inner_hid is None:
            d_inner_hid = d_hid
        self.w_1 = nn.Conv1d(d_hid, d_inner_hid, 1)  # position-wise
        self.w_2 = nn.Conv1d(d_inner_hid, d_hid, 1)  # position-wise
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x):
        output = self.relu(self.w_1(x.transpose(1, 2)))
        output = self.w_2(output).transpose(2, 1)
        output = self.dropout(output)
        return output

In [290]:
class POSEmbeddings(nn.Module):
    """
        构建位置编码
    """

    def __init__(self, config):
        super().__init__()      
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.position_embedding_type = getattr(config, "position_embedding_type", "absolute")
        self.register_buffer("position_ids", torch.arange(config.max_position_embeddings).expand((1, -1)))

    def forward(self, inputs_embeds=None, position_ids=None, past_key_values_length=0):
        input_shape = inputs_embeds.size()[:-1]
        seq_length = input_shape[1]
        embeddings = inputs_embeds
        if self.position_embedding_type == "absolute":
            position_embeddings = self.position_embeddings(position_ids)
            embeddings += position_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

In [260]:
class HAN_layer(nn.Module):
    '''
        用于屏蔽PAD句子的动态多层模型
    '''
    def __init__(self,model):
        super(HAN_layer,self).__init__()
        self.model = model
    def att_mask(self,x,max_len):
        return x*[1]+(max_len-x)*[0]
    def forward(self,x,x_paraghs,x_words):
        x_packed_paraghs = pack_padded_sequence(
                                                x,
                                                lengths =x_paraghs.tolist(),
                                                batch_first = True,
                                                enforce_sorted = False
                                               )
        x_packed_paraghs_words = pack_padded_sequence(
                                                        x_words,
                                                        lengths = x_paraghs.tolist(),
                                                        batch_first = True,
                                                        enforce_sorted = False
                                                    )
        mask = torch.tensor([self.att_mask(int(i),x.shape[-1]) for i in x_packed_paraghs_words.data]).to(device)
        z = self.model(x_packed_paraghs.data,attention_mask = mask)[1]
        
        sentences,x_paraghs = pad_packed_sequence(
                                                    PackedSequence(data = z,
                                                                    batch_sizes=x_packed_paraghs.batch_sizes,
                                                                    sorted_indices=x_packed_paraghs.sorted_indices,
                                                                    unsorted_indices=x_packed_paraghs.unsorted_indices
                                                                  ),
                                                    batch_first = True
                                                    )
        return sentences,x_paraghs

In [291]:
class GCN(nn.Module):
    '''
        定义GAT模型
    '''
    def __init__(self, embedding_dim = 768 ,mem_dim = 768, num_layers = 1, dropout = 0.01):
        super(GCN, self).__init__()
        self.mem_dim = mem_dim
        self.num_layers = num_layers
        self.dropout = dropout
        
        self.W = nn.ModuleList()
        for i in range(self.num_layers):
            input_dim = embedding_dim if i == 0 else self.mem_dim
            self.W.append(nn.Linear(input_dim,self.mem_dim))
        
        self.a = nn.ParameterList()
        for i in range(self.num_layers):
            input_dim = embedding_dim if i == 0 else self.mem_dim
            self.a.append(nn.Parameter(torch.empty(size=(input_dim, 1))))
            
        for i in range(self.num_layers):
            nn.init.xavier_uniform_(self.a[i].data,gain = 1.414)
            
        self.leakyrelu = nn.LeakyReLU()
    
    def gat_fun(self,A,Wx,a):
        def _prepare_attentional_mechanism_input(Wx):
            batch_size = Wx.size(0)
            N = Wx.size(1)
            out_feature = Wx.size(2)
            Wx_repeated_in_chunks = Wx.repeat_interleave(N, dim=1)
            Wx_repeated_alternating = Wx.repeat(1,N,1)
            all_combinations_matrix = torch.cat([Wx_repeated_in_chunks, Wx_repeated_alternating], dim=2)
            return all_combinations_matrix.view(batch_size,N,N,2*out_feature)
        
        def __attentional_mechanism_input_dot(Wx):
            WxT = Wx.permute(0,2,1)
            e = torch.bmm(Wx,WxT)
            return e
                                            
        def fun(A,Wx,a):
            e = self.leakyrelu(__attentional_mechanism_input_dot(Wx))
            zero_vec = -9e15*torch.ones_like(e)
            attention = torch.where(A>0,e,zero_vec)
            attention = F.softmax(attention,dim = -1)
            attention = F.dropout(attention,self.dropout,training = self.training)
            h_prime = torch.matmul(attention,Wx)
            return h_prime
                                            
        h_prime = fun(A,Wx,a)
        return h_prime
    
    def encode_with_gcn(self,adj,gcn_inputs):
        for l in range(self.num_layers):
            Wx = self.W[l](gcn_inputs)
            AWx = self.gat_fun(adj,Wx,self.a[l])
            gAWx = F.elu(AWx)

            gcn_inputs = self.gcn_drop(gAWx) if l < self.num_layers - 1 else gAWx
        
        return gcn_inputs
    
    def forward(self,inputs,adj):
        gcn_inputs = inputs
        gcn_out = self.encode_with_gcn(adj,gcn_inputs)
        
        return gcn_out

In [329]:
class CITSM(nn.Module):
    '''
        构造整体模型
    '''
    def __init__(self, config, bert, embedding_dim = 768, mem_dim = 768, num_layers = 1, dropout = 0.01, num_label = 2):
        super(CITSM,self).__init__()
      #  self.bert = bert
        self.p_han = HAN_layer(bert)
     #   self.w_han = HAN_layer(self.bert,opt)
        for p in self.p_han.parameters():
            p.requires_grad_(False)
        
        self.GAT = GCN(embedding_dim, mem_dim = mem_dim, num_layers = 1, dropout = 0.01 )
        
        self.pos_encoding = POSEmbeddings(config)
        
        

        self.ffn = PositionwiseFeedForward(embedding_dim,dropout = dropout)
        
        
        self.linear = nn.Linear(mem_dim, num_label)
        
    def inputs_to_tree_rep(self, heads, maxlen, directed = False, loop = True):
        '''
            构造batch邻接矩阵
            放在模型里更好一点
        '''
        trees = [head_to_tree(heads[i]) for i in range(len(heads))]
        adj = [tree_to_adj(maxlen, tree, directed, self_loop=loop).reshape(1, maxlen, maxlen) for tree in trees]
        adj = np.concatenate(adj,axis = 0)
        adj = torch.from_numpy(adj)
      
        Trees = torch.LongTensor([t.idx for t in trees])
        return adj, Trees
    
    
    def forward(self,inputs):
        
        input_ids = inputs[0]#应该是句子切片
        heads = inputs[1]
        max_paraghs = input_ids.shape[1]
        
        x_paraghs = torch.sum(input_ids!=0,dim = 1)
        x_paraghs = x_paraghs[:,0].cpu()
        x_words = torch.sum(input_ids!=0,dim=-1)
        p_h,_ = self.p_han(input_ids,x_paraghs,x_words)
        
        pos_ids = torch.arange(p_h.shape[1]).unsqueeze(0).expand(p_h.shape[0],max_paraghs)
        pos_out = self.pos_encoding(p_h, pos_ids)
        
        adj, trees = self.inputs_to_tree_rep(heads, max_paraghs)

        gcn_out = self.GAT(pos_out, adj)
        
        root_out = torch.cat([g[t].unsqueeze(0) for g, t in zip(gcn_out, trees)],dim = 0)
        
        out = self.linear(root_out)
        return out

In [330]:
def evaluate(model, data_iter, out_cases = False):
    model.eval()
    loss_total = 0
    predict_all = np.array([], dtype=int)
    labels_all = np.array([], dtype=int)
    with torch.no_grad():
        for i, (batch, label) in enumerate(data_iter):
            outputs = model(batch.to(deivce))
            loss = F.cross_entropy(outputs, label.to(device))
            loss_total += loss
            labels = labels.data.cpu().numpy()
            predict = torch.max(outputs,dim = 1)[1].cpu().numpy()
            labels_all = np.append(labels_all, labels)
            predict_all = np.append(predict_all, predict)
    
    f1 = metrics.f1_score(labels_all, predict_all, labels = [0,1], average = 'macro')
    acc = metrics.accuracy_score(labels_all, predict_all)
    return acc, loss_total / len(data_iter),f1

In [331]:
def train(model, train_iter, test_iter, learning_rate, num_epochs, require_improvement, save_path, device):
    start_time = time.time()
    model.train()
    model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]
    
    optimizer = torch.optim.Adam(optimizer_grouped_parameters, lr = learning_rate)
    
    total_batch = 0
    best_acc = 0
    last_improve = 0
    flag = False
    
    num_correct = 0
    num_loss = 0
    num_train = 0
    
    for epoch in range(num_epochs):
        print('Epoch [{}/{}]'.format(epoch + 1, num_epochs))
        for i,(batch, labels) in enumerate(train_iter):
            optimizer.zero_grad()
            outputs = model(batch.to(device))
            loss = F.cross_entropy(outputs, labels.to(device))
            
            num_train += len(labels)
            num_correct += torch.sum((torch.max(outputs,dim = 1)[1] == labels).type(torch.int)).item()
            num_loss += loss
             
            loss.backward()
            optimizer.step()
            total_batch += 1
            
            if total_batch % 10 == 0:
                
                dev_acc, dev_loss, dev_f1 = evaluate(model, test_iter)
                if dev_acc > best_acc:
                    best_acc = dev_acc
                    save_model(model, save_path)
                    
                    improve = '*'
                    last_improve = total_batch
                else:
                    improve = ''
                
                time_dif = get_time_dif(start_time)
                msg = 'Iter: {0:>6},  Train Loss: {3:>5.2},  Train Acc: {2:>6.2%},  Val Loss: {3:>5.2},  Val Acc: {4:>6.4%}, Val F1:{5:>6.4%} Time: {6} {7}'
                
                train_acc, train_loss = num_correct/num_train, num_loss/num_train
                print(msg.format(total_batch, train_loss, train_acc, dev_loss, dev_acc, dev_f1,time_dif, improve))
                
                model.train()
            
            if total_batch - last_improve > require_improvement:
                # 验证集loss超过1000batch没下降，结束训练
                print("No optimization for a long time, auto-stopping...")
                flag = True
                break
            break
        if flag:
            break
        break

In [255]:
if __name__ == '__main__':
    torch.cuda.empty_cache()#保证nvidia-smi中的可以显示被释放的显存
    np.random.seed(1)
    torch.manual_seed(1)
    torch.cuda.manual_seed_all(1)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    tokenizer = Tokenizer4bert(20, 'bert-base-chinese')
    bert = BertModel.from_pretrained('bert-base-chinese')
    bert_config = bert.config()
    
    start_time = time.time()
    print('Loading data...')
    train_path = '../Data/Dataset/Dataset/train_dependery.txt'
    test_path = '../Data/Dataset/Dataset/test_dependery.txt'
    trainset, train_max = load_data(tokenizer, path = train_path, pad_size = 20)
    testset, test_max = load_data(tokenizer, path = test_path, pad_size = 20)
    max_paragh = max(train_path, test_path)
    
    train_iter = DatasetIterater(trainset, batch_size = 16)
    test_iter = DatasetIterater(testset, batch_size = 16)
    
    time_dif = get_time_dif(start_time)
    print('Time usage:', time_dif)
    
    
    model = CISTM(bert_config, bert)
  #  train(model, train_iter, test_iter, 3e-5, 10, 1000, './', device)

In [83]:
x = torch.rand(3,4)
y = torch.zeros(3,4)

In [84]:
torch.stack([x,y],dim = 1)

tensor([[[0.7044, 0.0384, 0.9924, 0.9002],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.4473, 0.6129, 0.6466, 0.4315],
         [0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.3482, 0.4863, 0.9620, 0.0876],
         [0.0000, 0.0000, 0.0000, 0.0000]]])